# Evaluate two CAD objects

In [225]:
import cadquery as cq
import numpy as np
from scipy.spatial import KDTree
import trimesh
from typing import Callable, Optional
import ot

## Utilities

In [226]:
def sample_surface_points(
    obj: trimesh.Trimesh, num_samples: int = 1000, seed: int = 420
) -> np.ndarray:
    return trimesh.sample.sample_surface(obj, num_samples, seed=seed)[0]


def get_vertices(obj: trimesh.Trimesh, max_points: Optional[int] = None) -> np.ndarray:
    vertices = obj.vertices
    if max_points is not None and max_points > 0 and len(vertices) > max_points:
        indices = np.random.choice(len(vertices), max_points, replace=False)
        return vertices[indices]
    return vertices

## Setup

In [227]:
# These can be modified rather than hardcoding values for each dimension.
length = 80.0  # Length of the block
height = 60.0  # Height of the block
thickness = 10.0  # Thickness of the block
center_hole_dia = 22.0  # Diameter of center hole in block

box_cq = (
    cq.Workplane("XY").box(length, height, thickness).faces(">Z").workplane().hole(0)
)
box_cq.export("./box.stl")
holed_box_cq = (
    cq.Workplane("XY")
    .box(length, height, thickness)
    .faces(">Z")
    .workplane()
    .hole(center_hole_dia)
)
_ = holed_box_cq.export("./holed_box.stl")

In [228]:
holed_box = trimesh.load_mesh("./holed_box.stl")
holed_box2 = trimesh.load_mesh("./holed_box.stl")
box = trimesh.load_mesh("./box.stl")
sphere = trimesh.creation.uv_sphere(radius=1.0)
cube = trimesh.creation.box(extents=(1, 1, 1))

## Metrics

In [229]:
def eval_wrapper(error_value: float = 0.0, precision: int = 5, debug: bool = True):
    def decorator(function):
        def inner_wrapper(*args, **kwargs):
            try:
                return float(np.round(function(*args, **kwargs), decimals=precision))
            except Exception as e:
                if debug:
                    raise e
                return error_value

        return inner_wrapper

    return decorator

### Intersection Over Union

In [230]:
# @eval_wrapper()
# def iou(obj1: trimesh.Trimesh, obj2: trimesh.Trimesh) -> float:
#     intersection_volume = 0
#     for gt_mesh_i in obj1.split():
#         for pred_mesh_i in obj2.split():
#             intersection = gt_mesh_i.intersection(pred_mesh_i)
#             volume = intersection.volume if intersection is not None else 0
#             intersection_volume += volume

#     gt_volume = sum(m.volume for m in obj1.split())
#     pred_volume = sum(m.volume for m in obj2.split())
#     union_volume = gt_volume + pred_volume - intersection_volume
#     assert union_volume > 0
#     return intersection_volume / union_volume

In [231]:
@eval_wrapper()
def iou(obj1: trimesh.Trimesh, obj2: trimesh.Trimesh) -> float:
    # Aim: Maximization
    # Range: 0-1
    intersection_volume = trimesh.boolean.intersection(
        [obj1, obj2], check_volume=False
    ).volume
    union_volume = trimesh.boolean.union([obj1, obj2], check_volume=False).volume

    if intersection_volume == 0 or union_volume == 0:
        return 0.0
    return intersection_volume / union_volume

In [232]:
@eval_wrapper()
def voxel_iou(obj1: trimesh.Trimesh, obj2: trimesh.Trimesh, resolution: int = 32):
    # Aim: Maximization
    # Range: 0-1
    v1 = obj1.voxelized(pitch=obj1.scale / resolution).matrix
    v2 = obj2.voxelized(pitch=obj2.scale / resolution).matrix

    min_shape = np.minimum(v1.shape, v2.shape)
    v1 = v1[: min_shape[0], : min_shape[1], : min_shape[2]]
    v2 = v2[: min_shape[0], : min_shape[1], : min_shape[2]]

    intersection = np.logical_and(v1, v2).sum()
    union = np.logical_or(v1, v2).sum()
    return intersection / union if union != 0 else 0.0

### Generalized IoU

https://giou.stanford.edu/

In [233]:
# @eval_wrapper()
# def giou(obj1: trimesh.Trimesh, obj2: trimesh.Trimesh):
#     # Aim: Maximization
#     # Range: 0-1
#     if not obj1.is_watertight or not obj2.is_watertight:
#         return 0.0

#     inter = trimesh.boolean.intersection([obj1, obj2],check_volume=False)
#     vol_inter = inter.volume if inter is not None else 0.0

#     uni = trimesh.boolean.union([obj1, obj2], check_volume=False)
#     vol_union = uni.volume if uni is not None else obj1.volume + obj2.volume - vol_inter


#     all_vertices = np.vstack((obj1.vertices, obj2.vertices))
#     hull = trimesh.convex.convex_hull(all_vertices)
#     vol_hull = hull.volume

#     iou = vol_inter / vol_union if vol_union > 0 else 0.0
#     giou = iou - ((vol_hull - vol_union) / vol_hull if vol_hull > 0 else 0.0)

#     return giou


# giou(holed_box, sphere)

### Chamfer Distance

In [234]:
@eval_wrapper()
def inverse_chamfer_distance(
    obj1: trimesh.Trimesh, obj2: trimesh.Trimesh, num_samples: int = 5000
):
    # Aim: Maximization
    # Range: 0-1 (ONLY when scaling to unit scaled meshes)
    points1 = sample_surface_points(obj1, num_samples)
    points2 = sample_surface_points(obj2, num_samples)

    gt_distance, _ = KDTree(points1).query(points2, k=1)
    pred_distance, _ = KDTree(points2).query(points1, k=1)

    actual_distance = np.mean(np.square(gt_distance)) + np.mean(
        np.square(pred_distance)
    )

    # Following is only true on unit scaled meshes
    inverse_distance = 1.0 - actual_distance

    return inverse_distance

    # prox_query2 = trimesh.proximity.ProximityQuery(obj1)
    # prox_query1 = trimesh.proximity.ProximityQuery(obj2)

    # dist_1_to_2 = prox_query2.signed_distance(points1)
    # dist_2_to_1 = prox_query1.signed_distance(points2)
    # chamfer_dist = np.mean(dist_1_to_2**2) + np.mean(dist_2_to_1**2)

    # return chamfer_dist

In [235]:
@eval_wrapper()
def inverse_chamfer_distance_vertices(
    obj1: trimesh.Trimesh, obj2: trimesh.Trimesh, max_points: int = 5000
):
    # Aim: Maximization
    # Range: 0-1 (ONLY when scaling to unit scaled meshes)
    points1 = get_vertices(obj1, max_points)
    points2 = get_vertices(obj2, max_points)

    gt_distance, _ = KDTree(points1).query(points2, k=1)
    pred_distance, _ = KDTree(points2).query(points1, k=1)

    actual_distance = np.mean(np.square(gt_distance)) + np.mean(
        np.square(pred_distance)
    )

    # Following is only true on unit scaled meshes
    inverse_distance = 1.0 - actual_distance
    return inverse_distance

### Hausdorff Distance

https://en.wikipedia.org/wiki/Hausdorff_distance

In [236]:
@eval_wrapper()
def inverse_hausdorff_distance(
    obj1: trimesh.Trimesh, obj2: trimesh.Trimesh, num_samples: int = 5000
):
    # Aim: Maximization
    # Range: 0-1 (ONLY when scaling to unit scaled meshes)
    points1 = sample_surface_points(obj1, num_samples)
    points2 = sample_surface_points(obj2, num_samples)

    gt_distance, _ = KDTree(points1).query(points2, k=1)
    pred_distance, _ = KDTree(points2).query(points1, k=1)

    actual_distance = max(np.max(gt_distance), np.max(pred_distance))
    # Following is only true on unit scaled meshes
    inverse_distance = 1.0 - actual_distance
    return inverse_distance

In [237]:
@eval_wrapper()
def inverse_hausdorff_distance_vertices(
    obj1: trimesh.Trimesh, obj2: trimesh.Trimesh, max_points: int = 5000
):
    # Aim: Maximization
    # Range: 0-1 (ONLY when scaling to unit scaled meshes)
    points1 = get_vertices(obj1, max_points)
    points2 = get_vertices(obj2, max_points)

    gt_distance, _ = KDTree(points1).query(points2, k=1)
    pred_distance, _ = KDTree(points2).query(points1, k=1)

    actual_distance = max(np.max(gt_distance), np.max(pred_distance))
    # Following is only true on unit scaled meshes
    inverse_distance = 1.0 - actual_distance
    return inverse_distance

### Wasserstein distance

https://en.wikipedia.org/wiki/Wasserstein_metric

In [238]:
@eval_wrapper()
def inverse_wasserstain_distance(
    obj1: trimesh.Trimesh, obj2: trimesh.Trimesh, num_samples: int = 1000
) -> float:
    # Aim: Maximization
    # Range: 0-1 (ONLY when scaling to unit scaled meshes)
    points1 = sample_surface_points(obj1, num_samples)
    points2 = sample_surface_points(obj2, num_samples)

    a = np.ones((num_samples,)) / num_samples
    b = np.ones((num_samples,)) / num_samples

    cost_matrix = ot.dist(points1, points2, metric="sqeuclidean")

    actual_distance: float = ot.emd2(a, b, cost_matrix)  # type: ignore
    # Following is only true on unit scaled meshes
    inverse_distance = 1.0 - actual_distance
    return inverse_distance

### Volume/Surface Area Similarity

In [239]:
@eval_wrapper()
def volume_similarity(obj1: trimesh.Trimesh, obj2: trimesh.Trimesh) -> float:
    # Aim: Maximization
    # Range: 0-1
    # TODO: twice bigger and twice smaller give the same metric

    if not obj1.is_watertight or not obj2.is_watertight:
        return 0.0

    vol1 = float(obj1.volume)
    vol2 = float(obj2.volume)

    if vol1 + vol2 == 0:
        return 1.0
    return 1.0 - abs(vol1 - vol2) / max(vol1, vol2)

In [240]:
@eval_wrapper()
def area_similarity(obj1: trimesh.Trimesh, obj2: trimesh.Trimesh) -> float:
    # Aim: Maximization
    # Range: 0-1
    # TODO: twice bigger and twice smaller give the smae metric
    if not obj1.is_watertight or not obj2.is_watertight:
        return 0.0

    area1 = float(obj1.area)
    area2 = float(obj2.area)

    if max(area1, area2) == 0:
        return 1.0

    return 1.0 - abs(area1 - area2) / max(area1, area2)

### Bounding Box IoU

Faster approximation of IoU

In [241]:
@eval_wrapper()
def bbiou(obj1: trimesh.Trimesh, obj2: trimesh.Trimesh) -> float:
    bbox1 = obj1.bounds
    bbox2 = obj2.bounds

    min1, max1 = bbox1
    min2, max2 = bbox2

    inter_min = np.maximum(min1, min2)
    inter_max = np.minimum(max1, max2)

    if np.any(inter_min >= inter_max):
        return 0.0

    vol1 = np.prod(max1 - min1)
    vol2 = np.prod(max2 - min2)
    inter_vol = np.prod(inter_max - inter_min)

    union_vol = vol1 + vol2 - inter_vol

    if union_vol < 1e-9:
        return 0.0

    return inter_vol / union_vol

### Centroid distance

In [242]:
@eval_wrapper()
def inverse_centroid_distance(obj1: trimesh.Trimesh, obj2: trimesh.Trimesh) -> float:
    # Aim: Maximization
    # Range: 0-1 (ONLY when scaling to unit scaled meshes)
    actual_distance = float(np.linalg.norm(obj1.centroid - obj2.centroid))
    # Following is only true on unit scaled meshes
    inverse_distance = 1.0 - actual_distance
    return inverse_distance

### Inertia Similarity

https://en.wikipedia.org/wiki/Moment_of_inertia

In [243]:
@eval_wrapper()
def inertia_similarity(obj1: trimesh.Trimesh, obj2: trimesh.Trimesh) -> float:
    # Aim: Maximization
    # Range: 0-1
    i1 = obj1.moment_inertia
    i2 = obj2.moment_inertia
    diff = np.linalg.norm(i1 - i2)
    norm = np.linalg.norm(i1) + np.linalg.norm(i2)
    return float(1.0 - diff / norm if norm != 0 else 1.0)

## Evaluate

In [244]:
metrics_dict: dict[str, Callable[[trimesh.Trimesh, trimesh.Trimesh], float]] = {
    "iou": iou,
    # "giou": giou,
    "viou": voxel_iou,
    "cd": inverse_chamfer_distance,
    "cdv": inverse_chamfer_distance_vertices,
    "hd": inverse_hausdorff_distance,
    "hdv": inverse_hausdorff_distance_vertices,
    "wd": inverse_wasserstain_distance,
    "vs": volume_similarity,
    "as": area_similarity,
    # "bbiou": bbiou,
    "ctd": inverse_centroid_distance,
    "is": inertia_similarity,
}


def transform(obj: trimesh.Trimesh) -> trimesh.Trimesh:
    center = (obj.bounds[0] + obj.bounds[1]) / 2.0
    obj = obj.apply_translation(-center)
    extent = np.max(obj.extents)
    if extent > 1e-7:
        obj = obj.apply_scale(1.0 / extent)
    return obj.apply_transform(
        trimesh.transformations.translation_matrix([0.5, 0.5, 0.5])
    )


def evaluate(
    obj1: trimesh.Trimesh,
    obj2: trimesh.Trimesh,
    metrics_dict: dict[
        str, Callable[[trimesh.Trimesh, trimesh.Trimesh], float]
    ] = metrics_dict,
) -> dict[str, float]:
    transformed_obj1 = transform(obj1.copy())
    transformed_obj2 = transform(obj2.copy())
    return {
        name: metric_fn(transformed_obj1, transformed_obj2)
        for name, metric_fn in metrics_dict.items()
    }

## Experiments

In [245]:
# evaluate(holed_box, holed_box2)

In [246]:
evaluate(holed_box, box)

{'iou': 0.92084,
 'viou': 0.91509,
 'cd': 0.9995,
 'cdv': 0.75154,
 'hd': 0.87124,
 'hdv': 0.44954,
 'wd': 0.99742,
 'vs': 0.92084,
 'as': 0.99445,
 'ctd': 1.0,
 'is': 0.99689}

In [247]:
evaluate(cube, sphere)

{'iou': 0.52142,
 'viou': 0.10393,
 'cd': 0.96337,
 'cdv': 0.59547,
 'hd': 0.63652,
 'hdv': 0.29289,
 'wd': 0.96726,
 'vs': 0.52142,
 'as': 0.52251,
 'ctd': 1.0,
 'is': 0.47559}